
# Dataset analysis with Amazon SageMaker Processing Jobs using Apache Spark


Typically a machine learning (ML) process consists of few steps. First, gathering data with various ETL jobs, then pre-processing the data, featurizing the dataset by incorporating standard techniques or prior knowledge, and finally training an ML model using an algorithm.

Often, distributed data processing frameworks such as Spark are used to pre-process data sets in order to prepare them for training. In this notebook we'll use Amazon SageMaker Processing, and leverage the power of Spark in a managed SageMaker environment to run our preprocessing workload. Then, we'll take our preprocessed dataset and train a regression model using XGBoost.

In [88]:
!pip install pandas==1.0.2

tensorflow 2.0.0 requires opt-einsum>=2.3.2, which is not installed.
tensorflow 2.0.0 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
tensorflow 2.0.0 has requirement numpy<2.0,>=1.16.0, but you'll have numpy 1.14.3 which is incompatible.
awscli 1.18.20 has requirement botocore==1.15.20, but you'll have botocore 1.15.23 which is incompatible.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [89]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()

## Using Amazon SageMaker Processing to execute a SparkML job

### Build a Spark container for running the preprocessing job

An example Spark container is included in the `./container` directory of this example. The container handles the bootstrapping of all Spark configuration, and serves as a wrapper around the `spark-submit` CLI. At a high level the container provides:
* A set of default Spark/YARN/Hadoop configurations
* A bootstrapping script for configuring and starting up Spark master/worker nodes
* A wrapper around the `spark-submit` CLI to submit a Spark application


After the container build and push process is complete, use the Amazon SageMaker Python SDK to submit a managed, distributed Spark application that performs our dataset preprocessing.

Build the example Spark container.

In [90]:
!cat container/Dockerfile

FROM openjdk:8-jre-slim

RUN apt-get update
RUN apt-get install -y curl unzip python3 python3-setuptools python3-pip python-dev python3-dev python-psutil
RUN pip3 install py4j psutil==5.6.5 numpy==1.17.4
RUN apt-get clean
RUN rm -rf /var/lib/apt/lists/*

# http://blog.stuart.axelbrooke.com/python-3-on-spark-return-of-the-pythonhashseed
ENV PYTHONHASHSEED 0
ENV PYTHONIOENCODING UTF-8
ENV PIP_DISABLE_PIP_VERSION_CHECK 1

# Install Hadoop
ENV HADOOP_VERSION 3.0.0
ENV HADOOP_HOME /usr/hadoop-$HADOOP_VERSION
ENV HADOOP_CONF_DIR=$HADOOP_HOME/etc/hadoop
ENV PATH $PATH:$HADOOP_HOME/bin
RUN curl -sL --retry 3 \
  "http://archive.apache.org/dist/hadoop/common/hadoop-$HADOOP_VERSION/hadoop-$HADOOP_VERSION.tar.gz" \
  | gunzip \
  | tar -x -C /usr/ \
 && rm -rf $HADOOP_HOME/share/doc \
 && chown -R root:root $HADOOP_HOME

# Install Spark
ENV SPARK_VERSION 2.4.5
ENV SPARK_PACKAGE spark-${SPARK_VERSION}-bin-without-hadoop
ENV SPARK_HOME /usr/spark-${SPARK_VERSION}
ENV SPARK_DIST_CLASSPATH="$HADOOP_H

In [91]:
docker_repo = 'amazon-reviews-spark-analyzer'
docker_tag = 'latest'

In [92]:
!docker build -t $docker_repo:$docker_tag -f container/Dockerfile ./container

Sending build context to Docker daemon  3.022MB
Step 1/33 : FROM openjdk:8-jre-slim
 ---> 9c82c74fbc96
Step 2/33 : RUN apt-get update
 ---> Using cache
 ---> ca0a6099c443
Step 3/33 : RUN apt-get install -y curl unzip python3 python3-setuptools python3-pip python-dev python3-dev python-psutil
 ---> Using cache
 ---> cc1fd71bd88c
Step 4/33 : RUN pip3 install py4j psutil==5.6.5 numpy==1.17.4
 ---> Using cache
 ---> 376699e73ced
Step 5/33 : RUN apt-get clean
 ---> Using cache
 ---> 1ea11fb14632
Step 6/33 : RUN rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> ea880623fd34
Step 7/33 : ENV PYTHONHASHSEED 0
 ---> Using cache
 ---> 7d1b53453a5e
Step 8/33 : ENV PYTHONIOENCODING UTF-8
 ---> Using cache
 ---> 12cfee88f392
Step 9/33 : ENV PIP_DISABLE_PIP_VERSION_CHECK 1
 ---> Using cache
 ---> 8010c768a0e5
Step 10/33 : ENV HADOOP_VERSION 3.0.0
 ---> Using cache
 ---> 0e10f67d8992
Step 11/33 : ENV HADOOP_HOME /usr/hadoop-$HADOOP_VERSION
 ---> Using cache
 ---> 19f1a57f792c
Step 12/33 : ENV HADOOP

Create an Amazon Elastic Container Registry (Amazon ECR) repository for the Spark container and push the image.

In [93]:
import boto3
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account_id, region, docker_repo, docker_tag)
print(image_uri)

835319576252.dkr.ecr.us-east-1.amazonaws.com/amazon-reviews-spark-analyzer:latest


Create ECR repository and push docker image

In [94]:
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [95]:
!aws ecr describe-repositories --repository-names $docker_repo || aws ecr create-repository --repository-name $docker_repo

{
    "repositories": [
        {
            "repositoryArn": "arn:aws:ecr:us-east-1:835319576252:repository/amazon-reviews-spark-analyzer",
            "registryId": "835319576252",
            "repositoryName": "amazon-reviews-spark-analyzer",
            "repositoryUri": "835319576252.dkr.ecr.us-east-1.amazonaws.com/amazon-reviews-spark-analyzer",
            "createdAt": 1581738843.0,
            "imageTagMutability": "MUTABLE",
            "imageScanningConfiguration": {
                "scanOnPush": false
            }
        }
    ]
}


In [96]:
!docker tag $docker_repo:$docker_tag $image_uri

In [97]:
!docker push $image_uri

The push refers to repository [835319576252.dkr.ecr.us-east-1.amazonaws.com/amazon-reviews-spark-analyzer]

301d1115: Preparing 
268138d8: Preparing 
35559781: Preparing 
6b36b7a4: Preparing 
782557b4: Preparing 
8eb2663a: Preparing 
5ae17e8e: Preparing 
89b8c28a: Preparing 
d604f04b: Preparing 
a936c4d8: Preparing 
f855c32d: Preparing 
964f7673: Preparing 
0d7e7b4a: Preparing 
a6e6c92c: Preparing 
01d1115: Pushed       3MB/2.995MB5Alatest: digest: sha256:108967c05fc93e06543429632759c80d6f49ac87b4d184a37bf0eacef7113bd5 size: 3472


### Run the preprocessing job using Amazon SageMaker Processing

Next, use the Amazon SageMaker Python SDK to submit a processing job. Use the Spark container that was just built, and a SparkML script for preprocessing in the job configuration.

Review the Spark preprocessing script.

In [98]:
cat preprocess-deequ.py

from __future__ import print_function
from __future__ import unicode_literals

import time
import sys
import os
import shutil
import csv

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

def main():
    args_iter = iter(sys.argv[1:])
    args = dict(zip(args_iter, args_iter))
    
    # Retrieve the args and replace 's3://' with 's3a://' (used by Spark)
    s3_input_data = args['s3_input_data'].replace('s3://', 's3a://')
    print(s3_input_data)
    s3_output_analyze_data = args['s3_output_analyze_data'].replace('s3://', 's3a://')
    print(s3_output_analyze_data)
    
    spark = SparkSession.builder \
        .appName("Amazon_Reviews_Spark_Analyzer") \
        .getOrCreate()

    # Invoke Main from preprocess-deequ.jar
    getattr(spark._jvm.SparkAmazonReviewsAnalyzer, "run")(s3_input_data, s3_output_analyze_data)

if __name__ == "__main__":
    main()


In [99]:
from sagemaker.processing import ScriptProcessor

processor = ScriptProcessor(base_job_name='spark-amazon-reviews-analyzer',
                            image_uri=image_uri,
                            command=['/opt/program/submit'],
                            role=role,
                            instance_count=10, # instance_count needs to be > 1 or you will see the following error:  "INFO yarn.Client: Application report for application_ (state: ACCEPTED)"
                            instance_type='ml.r5.8xlarge',
#                            max_runtime_in_seconds=600,
                            env={
                                'mode': 'jar',
                                'main_class': 'Main'
                            })

In [100]:
# Inputs
s3_input_data = 's3://{}/amazon-reviews-pds/tsv/'.format(bucket)
print(s3_input_data)

s3://sagemaker-us-east-1-835319576252/amazon-reviews-pds/tsv/


In [101]:
!aws s3 ls $s3_input_data

2020-03-18 20:00:07  253570168 amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz
2020-03-18 20:00:06   18997559 amazon_reviews_us_Digital_Software_v1_00.tsv.gz
2020-03-18 20:00:11  506979922 amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz
2020-03-18 20:00:20   27442648 amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz


## Setup Output Data

In [102]:
from time import gmtime, strftime
timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

output_prefix = 'amazon-reviews-spark-analyzer-{}'.format(timestamp_prefix)
processing_job_name = 'amazon-reviews-spark-analyzer-{}'.format(timestamp_prefix)

print('Processing job name:  {}'.format(processing_job_name))

Processing job name:  amazon-reviews-spark-analyzer-2020-03-19-02-45-49


In [103]:
s3_output_analyze_data = 's3://{}/{}/analyze_data'.format(bucket, output_prefix)

print(s3_output_analyze_data)

s3://sagemaker-us-east-1-835319576252/amazon-reviews-spark-analyzer-2020-03-19-02-45-49/analyze_data


## Start the Spark Processing Job

_Notes on Invoking from Lambda:_
* However, if we use the boto3 SDK (ie. with a Lambda), we need to copy the `preprocess.py` file to S3 and specify the everything include --py-files, etc.
* We would need to do the following before invoking the Lambda:
     !aws s3 cp preprocess.py s3://<location>/sagemaker/spark-preprocess-reviews-demo/code/preprocess.py
     !aws s3 cp preprocess.py s3://<location>/sagemaker/spark-preprocess-reviews-demo/py_files/preprocess.py
* Then reference the s3://<location> above in the --py-files, etc.
* See Lambda example code in this same project for more details.

_Notes on not using ProcessingInput and Output:_
* Since Spark natively reads/writes from/to S3 using s3a://, we can avoid the copy required by ProcessingInput and ProcessingOutput (FullyReplicated or ShardedByS3Key) and just specify the S3 input and output buckets/prefixes._"
* See https://github.com/awslabs/amazon-sagemaker-examples/issues/994 for issues related to using /opt/ml/processing/input/ and output/
* If we use ProcessingInput, the data will be copied to each node (which we don't want in this case since Spark already handles this)

In [104]:
from sagemaker.processing import ProcessingOutput

processor.run(code='preprocess-deequ.py',
              arguments=['s3_input_data', s3_input_data,
                         's3_output_analyze_data', s3_output_analyze_data,
              ],
              # We need this dummy output to allow us to call 
              #    ProcessingJob.from_processing_name() later 
              #    to describe the job and poll for Completed status
              # See https://github.com/aws/sagemaker-python-sdk/issues/1341
              outputs=[
                  ProcessingOutput(s3_upload_mode='EndOfJob',
                                   output_name='dummy-output',
                                   source='/opt/ml/processing/output')
              ],
              logs=True,
              wait=False
)


Job Name:  spark-amazon-reviews-analyzer-2020-03-19-02-45-49-264
Inputs:  [{'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-835319576252/spark-amazon-reviews-analyzer-2020-03-19-02-45-49-264/input/code/preprocess-deequ.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'dummy-output', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-835319576252/spark-amazon-reviews-analyzer-2020-03-19-02-45-49-264/output/dummy-output', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]


In [105]:
preprocessing_job_description = processor.jobs[-1].describe()
print(preprocessing_job_description)

{'ProcessingInputs': [{'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-835319576252/spark-amazon-reviews-analyzer-2020-03-19-02-45-49-264/input/code/preprocess-deequ.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}], 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'dummy-output', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-835319576252/spark-amazon-reviews-analyzer-2020-03-19-02-45-49-264/output/dummy-output', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]}, 'ProcessingJobName': 'spark-amazon-reviews-analyzer-2020-03-19-02-45-49-264', 'ProcessingResources': {'ClusterConfig': {'InstanceCount': 10, 'InstanceType': 'ml.r5.8xlarge', 'VolumeSizeInGB': 30}}, 'StoppingCondition': {'MaxRuntimeInSeconds': 86400}, 'AppSpecification': {'ImageUri': '835319576252.dkr.ecr.us-east-1.amazonaws.com/amazon-reviews-spark-analy

In [106]:
processing_job_name = preprocessing_job_description['ProcessingJobName']

from IPython.core.display import display, HTML
display(HTML('<b>Review <a href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(region, processing_job_name)))

In [107]:
processing_job_status = preprocessing_job_description['ProcessingJobStatus']
if (processing_job_status in ['Completed', 'Stopped']):
    # TODO:  Do something interesting...
    print('Complete')
else:
    print(processing_job_status)

InProgress


## Please wait until the Processing Job Completes above


In [108]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(processing_job_name=processing_job_name,
                                                                            sagemaker_session=sagemaker_session)
running_processor.describe()

{'ProcessingInputs': [{'InputName': 'code',
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-835319576252/spark-amazon-reviews-analyzer-2020-03-19-02-45-49-264/input/code/preprocess-deequ.py',
    'LocalPath': '/opt/ml/processing/input/code',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'dummy-output',
    'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-835319576252/spark-amazon-reviews-analyzer-2020-03-19-02-45-49-264/output/dummy-output',
     'LocalPath': '/opt/ml/processing/output',
     'S3UploadMode': 'EndOfJob'}}]},
 'ProcessingJobName': 'spark-amazon-reviews-analyzer-2020-03-19-02-45-49-264',
 'ProcessingResources': {'ClusterConfig': {'InstanceCount': 10,
   'InstanceType': 'ml.r5.8xlarge',
   'VolumeSizeInGB': 30}},
 'StoppingCondition': {'MaxRuntimeInSeconds': 86400},
 'AppSpecification': {'ImageUri': '835319576252.dkr.ecr.us

In [109]:
from IPython.core.display import display, HTML
display(HTML('<b>Review <a href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Spark Job Has Completed</b>'.format(bucket, processing_job_name, region)))

#### Inspect the processed dataset
Take a look at a few rows of the transformed dataset to make sure the preprocessing was successful.

In [110]:
!aws s3 ls --recursive $s3_output_analyze_data/

# Load the output csv into a DataFrame and analyze
* constraint-checks/
* constraint-suggestions/
* dataset-metrics/
* success-metrics/

In [111]:
!aws s3 ls s3://$s3_output_analyze_data


Parameter validation failed:
Invalid bucket name "s3:": Bucket name must match the regex "^[a-zA-Z0-9.\-_]{1,255}$" or be an ARN matching the regex "^arn:(aws).*:s3:[a-z\-0-9]+:[0-9]{12}:accesspoint[/:][a-zA-Z0-9\-]{1,63}$"


In [112]:
!aws s3 cp --recursive s3://$s3_output_analyze_data ./amazon-reviews-spark-analyzer/

fatal error: Parameter validation failed:
Invalid bucket name "s3:": Bucket name must match the regex "^[a-zA-Z0-9.\-_]{1,255}$" or be an ARN matching the regex "^arn:(aws).*:s3:[a-z\-0-9]+:[0-9]{12}:accesspoint[/:][a-zA-Z0-9\-]{1,63}$"


In [113]:
import pandas as pd

# Change the filenames below to match the filenames above
TODO:  Programmatically retrieve the names (or just `read_csv('s3://...`)

In [114]:
pd.read_csv('./amazon-reviews-spark-analyzer/constraint-checks/part-00000-ba7d24dd-ec7d-4e06-9a12-9496fdbda252-c000.csv', delimiter='\t', header=0)

ImportError: cannot import name 'NotebookFormatter'

  check,check_level,check_status,constraint,constraint_status,constraint_message
0  Review Check,Error,Error,SizeConstraint(Size(N...                            
1  Review Check,Error,Error,"MaximumConstraint(Ma...                            
2  Review Check,Error,Error,UniquenessConstraint(...                            
3  Review Check,Error,Error,"ComplianceConstraint...                            
4  Review Check,Error,Error,"MinimumConstraint(Mi...                            
5  Review Check,Error,Error,"CompletenessConstrai...                            
6  Review Check,Error,Error,"CompletenessConstrai...                            

In [115]:
pd.read_csv('./amazon-reviews-spark-analyzer/constraint-suggestions/part-00000-21a096b1-e5dd-4190-aa05-0e93c9549d99-c000.csv', delimiter='\t', header=0)

ImportError: cannot import name 'NotebookFormatter'

                                             _1,_2,_3
0   review_id,'review_id' is not null,".isComplete...
1   customer_id,'customer_id' is not null,".isComp...
2   customer_id,'customer_id' has type Integral,"....
3   helpful_votes,'helpful_votes' has no negative ...
4   star_rating,'star_rating' is not null,".isComp...
5   star_rating,'star_rating' has no negative valu...
6   review_headline,'review_headline' has less tha...
7   product_id,'product_id' is not null,".isComple...
8   total_votes,'total_votes' is not null,".isComp...
9   product_category,"'product_category' has value...
10  product_parent,'product_parent' is not null,"....
11  product_parent,'product_parent' has type Integ...
12  vine,"'vine' has value range 'N', 'Y'",".isCon...
13  vine,'vine' has value range 'N' for at least 9...
14  marketplace,'marketplace' is not null,".isComp...
15  customer_id,'customer_id' has no negative valu...
16  review_date,'review_date' is not null,".isComp...
17  helpful_votes,'helpful_v

In [ ]:
pd.read_csv('./amazon-reviews-spark-analyzer/dataset-metrics/part-00000-9897ff58-4376-411d-a569-40de3d764da9-c000.csv', delimiter='\t', header=0)

In [ ]:
pd.read_csv('./amazon-reviews-spark-analyzer/success-metrics/part-00000-c42aeccd-d35d-44ed-8fa3-72d25f05a374-c000.csv', delimiter='\t', header=0)